# Projeto 1 - Ciência dos Dados

Nome: Luiz Ricardo Hardman Paranhos

Nome: Matheus Kwon

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [64]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [65]:
import os

filename = 'cheetos.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo cheetos.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [66]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,relevancia
0,@eaipaixao_ cheetos obv,1.0
1,@giovannapinto mano eu fui comprar 2 cheetos e...,1.0
2,@zabele__ só tenho cheetos,0.0
3,"@lilizvante cheetos, old\nhttps://t.co/atamf2woj7",0.0
4,@biaventura14 kkkkkkkkkkkkkkkkkkkkk cheetos ve...,1.0


In [67]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,relevancia
0,sou do tipo que quando tá conversando com algu...,0
1,att fresquinha de flaming hot cheetos!!!! http...,0
2,me entupindo de cheetos enquanto vejo tapas e ...,0
3,cheetos ou doritos https://t.co/caytqjylnp,0
4,mas quem colocou o cheetos na minha tml hoje ?,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

- Relevante: informação da qual a empresa pode ganhar dinheiro
- Irrelevante: resto

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [68]:
train['relevancia'] = train['relevancia'].astype('category')
train.relevancia.cat.categories = ['irrelevante', 'relevante']

In [69]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;/,_@¿]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def cleanup_enter(text):
    punctuation = '\n'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def link(text):
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\S+', '', text, flags=re.MULTILINE)
    return text

In [70]:
# String com todas as palavras
tweets_treinamento_raw = ''
for tweet in train.Treinamento:
    tweets_treinamento_raw = tweets_treinamento_raw + ' ' + tweet
tweets_treinamento = deEmojify(cleanup_enter(cleanup(link(tweets_treinamento_raw.lower()))))

In [71]:
# String com os relevantes
filtro_relevante_train = train.relevancia == 'relevante'
treinamento_relevante = train.loc[filtro_relevante_train,:]

tweets_relevantes = ''
for tweet in treinamento_relevante.Treinamento:
    tweets_relevantes = tweets_relevantes + ' ' + tweet

treinamento_relevante = deEmojify(cleanup_enter(cleanup(link(tweets_relevantes.lower()))))

In [72]:
# String com os irrelevantes
filtro_irrelevante_train = train.relevancia == 'irrelevante'
treinamento_irrelevante = train.loc[filtro_irrelevante_train,:]

tweets_irrelevantes = ''
for tweet in treinamento_irrelevante.Treinamento:
    tweets_irrelevantes = tweets_irrelevantes + ' ' + tweet

    
treinamento_irrelevante = deEmojify(cleanup_enter(cleanup(tweets_irrelevantes.lower())))

In [73]:
# Criando lista e séries a partir das strings
tweets_treinamento_lista = tweets_treinamento.split()
treinamento_relevante_lista = treinamento_relevante.split()
treinamento_irrelevante_lista = treinamento_irrelevante.split()

serie_treinamento = pd.Series(tweets_treinamento_lista)
serie_treinamento_relevante = pd.Series(treinamento_relevante_lista)
serie_treinamento_irrelevante = pd.Series(treinamento_irrelevante_lista)

In [74]:
tabela_treinamento = serie_treinamento.value_counts()
tabela_treinamento

cheetos    297
de         171
e          116
o           81
um          81
          ... 
harsh        1
tb           1
2020         1
tou          1
for          1
Length: 1236, dtype: int64

In [75]:
tabela_treinamento_relevante = serie_treinamento_relevante.value_counts()
tabela_treinamento_relevante

cheetos      151
de            92
e             48
um            39
eu            31
            ... 
funcional      1
trakinas       1
xau            1
agora          1
for            1
Length: 554, dtype: int64

In [85]:
stopwords = ['uma','em','mais','q','que','na','da','do','no','eu','de','e','o','um','a','com','pra']

In [86]:
# Removendo stop words da lista de todas as palavras
tweets_treinamento_stopwords_lista = []
for palavra in tweets_treinamento_lista:
    x = 0
    if palavra in stopwords:
        x = 1
    if x == 0:
        if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
            tweets_treinamento_stopwords_lista.append(palavra[:-3]+'r')
        elif palavra == 'n' or palavra == 'nao':
            treinamento_irrelevante_stopwords_lista.append('não')
        else:
            tweets_treinamento_stopwords_lista.append(palavra)
        

serie_treinamento_filtrado = pd.Series(tweets_treinamento_stopwords_lista)
tabela_treinamento_filtrado = serie_treinamento_filtrado.value_counts(True)
tabela_treinamento_filtrado.head(20)

cheetos      0.112202
é            0.022667
comer        0.014356
requeijão    0.011711
não          0.011711
doritos      0.009822
tem          0.009822
meu          0.008689
mas          0.007178
me           0.007178
só           0.006800
por          0.006045
gosto        0.005289
pq           0.004911
se           0.004911
melhor       0.004911
cheiro       0.004533
vontade      0.004533
minha        0.004533
vou          0.004533
dtype: float64

In [87]:
# Removendo stop words da lista de todas as palavras relevantes
treinamento_relevante_stopwords_lista = []
for palavra in treinamento_relevante_lista:
    x = 0
    if palavra in stopwords:
        x = 1
    if x == 0:
        if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
            treinamento_relevante_stopwords_lista.append(palavra[:-3]+'r')
        elif palavra == 'n' or palavra == 'nao':
            treinamento_irrelevante_stopwords_lista.append('não')
        else:
            treinamento_relevante_stopwords_lista.append(palavra)

serie_treinamento_relevante_filtrado = pd.Series(treinamento_relevante_stopwords_lista)
tabela_treinamento_relevante_filtrado = serie_treinamento_relevante_filtrado.value_counts(True)
tabela_treinamento_relevante_filtrado.head(20)

cheetos       0.137273
é             0.027273
requeijão     0.025455
comer         0.021818
doritos       0.017273
gosto         0.010909
meu           0.010000
vontade       0.010000
melhor        0.010000
mas           0.009091
queria        0.009091
tem           0.008182
me            0.007273
só            0.007273
fandangos     0.007273
ou            0.006364
salgadinho    0.006364
sabor         0.006364
pq            0.006364
pacote        0.006364
dtype: float64

In [88]:
# Removendo stop words da lista de todas as palavras irrelevantes
treinamento_irrelevante_stopwords_lista = []
for palavra in treinamento_irrelevante_lista:
    x = 0
    if palavra in stopwords:
        x = 1
    if x == 0:
        if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
            treinamento_irrelevante_stopwords_lista.append(palavra[:-3]+'r')
        elif palavra == 'n' or palavra == 'nao':
            treinamento_irrelevante_stopwords_lista.append('não')
        else:
            treinamento_irrelevante_stopwords_lista.append(palavra)

serie_treinamento_irrelevante_filtrado = pd.Series(treinamento_irrelevante_stopwords_lista)
tabela_treinamento_irrelevante_filtrado = serie_treinamento_irrelevante_filtrado.value_counts(True)
tabela_treinamento_irrelevante_filtrado.head(20)

cheetos    0.084994
não        0.023447
é          0.017585
tem        0.009965
por        0.008206
comer      0.008206
meu        0.007034
me         0.006448
cheiro     0.005862
vou        0.005862
só         0.005862
mas        0.005275
ele        0.005275
minha      0.004689
como       0.004689
se         0.004103
seu        0.004103
aqui       0.004103
tipo       0.004103
as         0.003517
dtype: float64

In [79]:
P_R = len(treinamento_relevante_stopwords_lista) / len(tweets_treinamento_stopwords_lista)
print('P(R):',P_R)
P_Rc = 1 - P_R
print('P(Rc):',P_Rc)
print(P_R + P_Rc)

P_RequeijaoDadoR = tabela_treinamento_relevante_filtrado['requeijão']
P_Requeijao = tabela_treinamento_filtrado['requeijão']
P_RDadoRequeijao = P_RequeijaoDadoR * P_R / P_Requeijao
print('P(R|Requeijao):',P_RDadoRequeijao)

P_RequeijaoDadoRc = tabela_treinamento_irrelevante_filtrado['requeijão']
P_RcDadoRequeijao = P_RequeijaoDadoRc * P_Rc / P_Requeijao
print('P(Rc|Requeijao):',P_RcDadoRequeijao)

# print(P_RequeijaoDadoRc)
# print(P_RequeijaoDadoR)
print(P_RDadoRequeijao+P_RcDadoRequeijao)


P(R): 0.41492537313432837
P(Rc): 0.5850746268656717
1.0
P(R|Requeijao): 0.903225806451613
P(Rc|Requeijao): 0.08786446756448811
0.9910902740161011


In [80]:
P_DoritosDadoR = tabela_treinamento_relevante_filtrado['doritos']
P_Doritos = tabela_treinamento_filtrado['doritos']
P_RDadoDoritos = P_DoritosDadoR * P_R / P_Doritos
print('P(R|Doritos):',P_RDadoDoritos)

P_DoritosDadoRc = tabela_treinamento_irrelevante_filtrado['doritos']
P_RcDadoDoritos = P_DoritosDadoRc * P_Rc / P_Doritos
print('P(Rc|Doritos):',P_RcDadoDoritos)

# print(P_DoritosDadoRc)
# print(P_DoritosDadoR)
print(P_RDadoDoritos+P_RcDadoDoritos)


P(R|Doritos): 0.7307692307692308
P(Rc|Doritos): 0.20952296111531782
0.9402921918845486


In [81]:
P_ComerDadoR = tabela_treinamento_relevante_filtrado['comer']
P_Comer = tabela_treinamento_filtrado['comer']
P_RDadoComer = P_ComerDadoR * P_R / P_Comer
print('P(R|Comer):',P_RDadoComer)

P_ComerDadoRc = tabela_treinamento_irrelevante_filtrado['comer']
P_RcDadoComer = P_ComerDadoRc * P_Rc / P_Comer
print('P(Rc|Comer):',P_RcDadoComer)

# print(P_ComerDadoRc)
# print(P_ComerDadoR)
print(P_RDadoComer+P_RcDadoComer)


P(R|Comer): 0.631578947368421
P(Rc|Comer): 0.33450156949989335
0.9660805168683144


In [82]:
P_CheiroDadoR = tabela_treinamento_relevante_filtrado['cheiro']
P_Cheiro = tabela_treinamento_filtrado['cheiro']
P_RDadoCheiro = P_CheiroDadoR * P_R / P_Cheiro
print('P(R|Cheiro):',P_RDadoCheiro)

P_CheiroDadoRc = tabela_treinamento_irrelevante_filtrado['cheiro']
P_RcDadoCheiro = P_CheiroDadoRc * P_Rc / P_Cheiro
print('P(Rc|Cheiro):',P_RcDadoCheiro)

# print(P_CheiroDadoRc)
# print(P_CheiroDadoR)
print(P_RDadoCheiro+P_RcDadoCheiro)


P(R|Cheiro): 0.16666666666666666
P(Rc|Cheiro): 0.7566106929164254
0.923277359583092


In [83]:
P_SalgadinhoDadoR = tabela_treinamento_relevante_filtrado['salgadinho']
P_Salgadinho = tabela_treinamento_filtrado['salgadinho']
P_RDadoSalgadinho = P_SalgadinhoDadoR * P_R / P_Salgadinho
print('P(R|Salgadinho):',P_RDadoSalgadinho)

P_SalgadinhoDadoRc = tabela_treinamento_irrelevante_filtrado['salgadinho']
P_RcDadoSalgadinho = P_SalgadinhoDadoRc * P_Rc / P_Salgadinho
print('P(Rc|Salgadinho):',P_RcDadoSalgadinho)

# print(P_SalgadinhoDadoRc)
# print(P_SalgadinhoDadoR)
print(P_RDadoSalgadinho+P_RcDadoSalgadinho)


P(R|Salgadinho): 0.7777777777777778
P(Rc|Salgadinho): 0.2017628514443801
0.9795406292221579


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [84]:
string = 'comendo'
print(string[:-4])

com
